# Negative Comment Classification Implementation


### Utils

In [1]:
# utils.py
# define some method will be used later
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F

def load_training_data(path='training_label.txt'):
    # read label if 'training_label' is in the filename
    if 'training_label' in path:
        with open(path, 'r',encoding = 'utf-8') as f:
            lines = f.readlines()
            lines = [line.strip('\n').split(' ') for line in lines]
        x = [line[2:] for line in lines]
        y = [line[0] for line in lines]
        return x, y
    else:
        with open(path, 'r',encoding = 'utf-8') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
        return x

def load_testing_data(path='testing_data'):
    with open(path, 'r',encoding = 'utf-8') as f:
        lines = f.readlines()
        X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
        X = [sen.split(' ') for sen in X]
    return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    # output > threshold => positive
    # output < threshold => negative
    # The threshold is set to be 0.5.
    outputs[outputs>=0.5] = 1 
    outputs[outputs<0.5] = 0
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

### Train Word to Vector

In [3]:
# w2v.py
# Train model for word embedding
import os
import numpy as np
import pandas as pd
import argparse
import smart_open
# This line resolve the bug when importing gensim.models
smart_open.open = smart_open.smart_open

from gensim.models import word2vec


def train_word2vec(x):
    # train word2vec model for word embedding
    model = word2vec.Word2Vec(x, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)
    return model

if __name__ == "__main__":
    print("loading training data ...")
    path_prefix = r'C:\Data\RNN_Data'
    train_path = path_prefix + r'/training_label.txt'
    train_nolabel_path = path_prefix + r'/training_nolabel.txt'
    train_x, y = load_training_data(train_path)
    train_x_no_label = load_training_data(train_nolabel_path)

    print("loading testing data ...")
    test_path = path_prefix + r'/testing_data.txt'
    test_x = load_testing_data(test_path)

    model = train_word2vec(train_x + train_x_no_label + test_x)
    #model = train_word2vec(train_x + test_x)
    
    print("saving model ...")
    # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
    model.save(os.path.join(path_prefix, 'w2v_all2.model'))

loading training data ...
loading testing data ...
saving model ...


### Data Preprocess

In [2]:
# preprocess.py
import os
import numpy as np
import pandas as pd
import argparse
from torch import nn
import smart_open
smart_open.open = smart_open.smart_open
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path="./w2v.model"):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # Read the pretrained Word2Vec model
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # add new word to embedding，and give it a randomly generated representation vector
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
    def make_embedding(self, load=True):
        print("Get embedding ...")
        # load pretrained Word2vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # Create word2idx  dictionary
        # Create idx2word  list
        # Create word2vector  list
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words #{}'.format(i+1), end='\r')
            #e.g. self.word2index['he'] = 1 
            #e.g. self.index2word[1] = 'he'
            #e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # Add "<PAD>" and "<UNK>" into embedding 
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    def pad_sequence(self, sentence):
        # ensure every sentence has same length
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self):
        # Transfer every word into index
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # padding
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        #  labels to tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)


### Dataset

In [3]:
# data.py
import torch
from torch.utils import data

class TwitterDataset(data.Dataset):
    """
    Expected data shape like:(data_num, data_len)
    Data can be a list of numpy array or a list of lists
    input data shape : (data_num, seq_len, feature_dim)
    
    __len__ will return the number of data
    """
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

### Model

In [4]:
# model.py
import torch
from torch import nn
class LSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super().__init__()
        # embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # fix embedding by setting fix_embedding True    
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        #nn.LSTM (input_size, hidden_size, num_layers, batch_first, dropout, bidirectional,proj_size) 
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)        
 
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(hidden_dim, 1),
                                         nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        #print(inputs.size())
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_size)
        x = x[:, -1, :] 
        x = self.classifier(x)
        return x

### Bi LSTM


In [5]:
# model.py
import torch
from torch import nn
class BiLSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super().__init__()
        # embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # fix embedding by setting fix_embedding True 
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True,bidirectional=True)        
 
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(hidden_dim*2, 1),
                                         nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # dimension of x => (batch, seq_len, hidden_size)
        # Last hidden state of LSTM is used 
        x = x[:, -1, :] 
        x = self.classifier(x)
        return x

### Self-Attention

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attn_Clf(nn.Module):
    def __init__(self, embedding, embedding_dim, dropout=0.1):
        super().__init__()
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        

        self.encoder_layer = nn.TransformerEncoderLayer(
          d_model=embedding_dim, dim_feedforward=350, nhead=2
        )
        #self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential( nn.Dropout(dropout),
                                         nn.Linear(embedding_dim, 1),
                                         nn.Sigmoid() )


    def forward(self, inputs):
        """
        args:
          mels: (batch size, length, 40)
        return:
          out: (batch size, n_spks)
        """
        inputs = self.embedding(inputs)
        inputs = inputs.permute(1, 0, 2)        
        x= self.encoder_layer(inputs)
        x = x.transpose(0, 1)
        x = x.mean(dim=1)        
        out = self.pred_layer(x)
        return out


In [24]:
z= X_train[1][None,:].to(device)
ml = Attn_Clf(embedding, embedding_dim=250, dropout=0.1)
ml = ml.to(device)

model = LSTM_Net(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device)

print(z.size())
print(ml(z).size())
print(ml(z))

print(z.size())
print(model(z).size())
print(model(z))

torch.Size([1, 20])
torch.Size([1, 1])
tensor([[0.6320]], device='cuda:0', grad_fn=<SigmoidBackward>)
torch.Size([1, 20])
torch.Size([1, 1])
tensor([[0.5926]], device='cuda:0', grad_fn=<SigmoidBackward>)


### Train

In [7]:
# train.py

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    model.train() 
    criterion = nn.BCELoss() 
    t_batch = len(train) 
    v_batch = len(valid) 
    optimizer = optim.Adam(model.parameters(), lr=lr)
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long)
            labels = labels.to(device, dtype=torch.float) 
            optimizer.zero_grad() 
            outputs = model(inputs) 
            outputs = outputs.squeeze() 
            loss = criterion(outputs, labels) 
            loss.backward() 
            optimizer.step() 
            correct = evaluation(outputs, labels)
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(
            	epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size), end='\r')
        print('\nTrain | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # validation
        model.eval() 
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) 
                labels = labels.to(device, dtype=torch.float) 
                outputs = outputs.squeeze() 
                loss = criterion(outputs, labels) 
                correct = evaluation(outputs, labels) 
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                #save the model if it has the best performance.
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/ckpt.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        model.train() # set model back to train mode

### Test

In [8]:
# test.py

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            outputs[outputs>=0.5] = 1 
            outputs[outputs<0.5] = 0 
            ret_output += outputs.int().tolist()
    
    return ret_output

### Main

In [9]:
# precheck if cuda works
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
# main.py
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
from sklearn.model_selection import train_test_split

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path_prefix = r'C:/Data/RNN_Data'
# Data path
train_with_label = os.path.join(path_prefix, 'training_label.txt')
train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join(path_prefix, 'w2v_all2.model') # 處理 word to vec model 的路徑

# Sentence Length、fix embedding、batch size、 epoch、learning rate 、model path
sen_len = 20
fix_embedding = True # fix embedding during training
batch_size = 128
epoch = 30
lr = 0.001
# model_dir = os.path.join(path_prefix, 'model/') # model directory for checkpoint model
model_dir = path_prefix # model directory for checkpoint model

print("loading data ...") 
train_x, y = load_training_data(train_with_label)
train_x_no_label = load_training_data(train_no_label)

#preprocess
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# model setting
model = BiLSTM_Net(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device)

# Split training data to validation and training set
X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]

# Create Data set
train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)

# Create DataLoader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 0)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)

# Start training
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)

loading data ...
Get embedding ...
loading word to vec model ...
get words #2341

C:\Users\user\anaconda3\envs\DLenv\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words #7184

get words #9583

get words #11888

...

get words #54160

get words #55777


total words: 55779
sentence count #199789sentence count #200000
start training, parameter total:14427451, trainable:482701

[ Epoch1: 1407/1407 ] loss:0.478 acc:17.969 
Train | Loss:0.49464 Acc: 75.223
Valid | Loss:0.44836 Acc: 78.702 
saving model with acc 78.702
-----------------------------------------------


C:\Users\user\anaconda3\envs\DLenv\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type BiLSTM_Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[ Epoch2: 1407/1407 ] loss:0.344 acc:21.875 
Train | Loss:0.43694 Acc: 79.530
Valid | Loss:0.43662 Acc: 79.459 
saving model with acc 79.459
-----------------------------------------------
[ Epoch3: 1407/1407 ] loss:0.352 acc:21.875 
Train | Loss:0.42157 Acc: 80.485
Valid | Loss:0.43248 Acc: 79.618 
saving model with acc 79.618
-----------------------------------------------
[ Epoch4: 1407/1407 ] loss:0.461 acc:18.750 
Train | Loss:0.40849 Acc: 81.176
Valid | Loss:0.42913 Acc: 80.056 
saving model with acc 80.056
-----------------------------------------------
[ Epoch5: 1407/1407 ] loss:0.314 acc:23.438 
Train | Loss:0.39660 Acc: 81.816
Valid | Loss:0.41723 Acc: 80.494 
saving model with acc 80.494
-----------------------------------------------
[ Epoch6: 1407/1407 ] loss:0.354 acc:20.312 
Train | Loss:0.38365 Acc: 82.520
Valid | Loss:0.41567 Acc: 80.683 
saving model with acc 80.683
-----------------------------------------------
[ Epoch7: 1407/1407 ] loss:0.524 acc:17.969 
Train | Lo

In [11]:
# This block is for convenient for only process training
epoch = 30
lr = 0.001

#model = BiLSTM_Net(embedding, embedding_dim=250, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
#model = model.to(device) 

model = Attn_Clf(embedding, embedding_dim=250, dropout=0.4)
model = model.to(device)

X_train, X_val, y_train, y_val = train_x[:180000], train_x[180000:], y[:180000], y[180000:]


train_dataset = TwitterDataset(X=X_train, y=y_train)
val_dataset = TwitterDataset(X=X_val, y=y_val)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 0)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)


training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)


start training, parameter total:14372601, trainable:427851

[ Epoch1: 1407/1407 ] loss:0.329 acc:21.875 
Train | Loss:0.47339 Acc: 77.189
Valid | Loss:0.46322 Acc: 77.966 
saving model with acc 77.966
-----------------------------------------------


C:\Users\user\anaconda3\envs\DLenv\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attn_Clf. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[ Epoch2: 1407/1407 ] loss:0.490 acc:19.531 
Train | Loss:0.45188 Acc: 78.584
Valid | Loss:0.45359 Acc: 78.349 
saving model with acc 78.349
-----------------------------------------------
[ Epoch3: 1407/1407 ] loss:0.338 acc:23.438 
Train | Loss:0.44524 Acc: 78.976
Valid | Loss:0.45391 Acc: 78.210 
-----------------------------------------------
[ Epoch4: 1407/1407 ] loss:0.367 acc:22.656 
Train | Loss:0.44007 Acc: 79.281
Valid | Loss:0.45095 Acc: 78.533 
saving model with acc 78.533
-----------------------------------------------
[ Epoch5: 1407/1407 ] loss:0.461 acc:20.312 
Train | Loss:0.43652 Acc: 79.439
Valid | Loss:0.46057 Acc: 78.722 
saving model with acc 78.722
-----------------------------------------------
[ Epoch6: 1407/1407 ] loss:0.363 acc:22.656 
Train | Loss:0.43270 Acc: 79.774
Valid | Loss:0.44450 Acc: 78.886 
saving model with acc 78.886
-----------------------------------------------
[ Epoch7: 1407/1407 ] loss:0.360 acc:19.531 
Train | Loss:0.42830 Acc: 79.921
Valid 

### Predict and Write to csv file

In [12]:
# Start prediction
print("loading testing data ...")
test_x = load_testing_data(testing_data)
preprocess = Preprocess(test_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = TwitterDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 0)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, 'ckpt.model'))
outputs = testing(batch_size, test_loader, model, device)

# write to csv file
tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
print("save csv ...")
tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
print("Finish Predicting")


loading testing data ...
Get embedding ...
loading word to vec model ...


C:\Users\user\anaconda3\envs\DLenv\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


get words #4895

get words #16568

get words #18976

...

get words #47258

get words #49596

get words #51921

get words #55777


total words: 55779
sentence count #200000
load model ...
save csv ...
Finish Predicting
